In [5]:
!pip install graphene
!pip install starlette_graphene3
!pip install uvicorn[standard]
!pip install fastapi

In [6]:
assert port in [9991, 9992, 9993, 9994]

NameError: name 'port' is not defined

In [7]:
import uvicorn
from multiprocessing import Process

servers = {}
_api_process = None

def start_api(app=None, port=9992, runNew=True):
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    assert port in [9991, 9992, 9993, 9994], f'port has unexpected value {port}'
    def run():
        uvicorn.run(app, port=port, host='0.0.0.0', root_path='')    
        
    _api_process = servers.get(port, None)
    if _api_process:
        _api_process.terminate()
        _api_process.join()
        del servers[port]
    
    if runNew:
        assert (not app is None), 'app is None'
        _api_process = Process(target=run, daemon=True)
        _api_process.start()
        servers[port] = _api_process

In [8]:
import graphene

class ExamGQL(graphene.ObjectType):
    id = graphene.ID()
    typ = graphene.String()
    exam_date = graphene.String()
    student_capacity = graphene.Int() 
    signed_students = graphene.List(UserGQL) #ForeignKey
    additional_information = graphene.String()
    subject_id = graphene.Int() #ForeignKey ID/Int?
    
class UserGQL(graphene.ObjectType):
    id = graphene.ID()
    name = graphene.String()
    surname = graphene.String()
    title = graphene.String()
    email = graphene.String()
    phone_number = graphene.String()
    id_study_group = graphene.ID() #ForeignKey
    

    #vztahy? presne definovat nazvy ("exams_examiners = ...")
    #groups = graphene.Field(graphene.List(lambda: GroupGQL))
    exams = graphene.Field(graphene.List(lambda: ExamGQL))
    #vztahy? neni nutne
    #def resolve_groups(parent, info):
     #   return parent.groups    
    
class SubjectGQL(graphene.ObjectType):
    id = graphene.ID()
    id_study_group = graphene.ID() #ForeignKey
    name = graphene.String()
    
class StudyGroupModel(graphene.ObjectType):
    id = graphene.ID()
    name = graphene.String()
    
class AuthorizationModel(graphene.ObjectType):
    id = graphene.ID()
    web_admin = graphene.Boolean()
    student = graphene.Boolean()
    examiner = graphene.Boolean()

class QueryGQL(graphene.ObjectType):
    user = graphene.Field(UserGQL, id = graphene.ID(required = True))
    

In [7]:
#from starlette.graphql import GraphQLApp
from starlette_graphene3 import GraphQLApp, make_graphiql_handler

import graphene
from fastapi import FastAPI

graphql_app = GraphQLApp(schema=graphene.Schema(query=QueryGQL), on_get=make_graphiql_handler())

app = FastAPI()#root_path='/api')

@app.get('/')
def hello():
    return {'hello': 'world'}

app.add_route('/gql/', graphql_app)
start_api(app=app, port=9992, runNew=True)

NameError: name 'start_api' is not defined

In [15]:
#ukonceni, uvoleneni portu
start_api(app=app, port=9992, runNew=False)

NameError: name 'app' is not defined